In [3]:
import pandas as pd
import numpy as np

In [24]:
def device_os_pre(df: pd.DataFrame) -> pd.DataFrame:
    shame_df = df.shape[0]
    column_uniq = []
    
    for column in df.columns:
        if len(df[column].unique()) == shame_df:
            column_uniq.append(column)
            break
    
    df_combinations = df[
    ['device_category', 
     'device_brand', 
     'device_os']
    ].drop_duplicates().dropna()
    
    df_combinations.drop(
        df_combinations
        .loc[df_combinations['device_os'] == '(not set)']
        .index, 
        inplace=True
    )

    df = df.merge(df_combinations, 
                       on=['device_category', 'device_brand'], 
                       how='left', 
                       suffixes=('', '_y')
                      )

    df_clear = df.drop('device_os', axis=1).dropna()

    df_finish = df_clear[~df_clear[column_uniq].duplicated()]
    
    # df_sessions = df[
    # (~df['device_os_y'].isna()) & (df[column_uniq].duplicated())]
    
    # df_finish = df.merge(df_sessions[column_uniq], 
    #                      on=column_uniq
    #                     )
    
    return df_finish

ga_sessions.utm_medium in ('organic',
'referral', '(none)') ===================== ORGANIC
everything else = no ORGANIC

In [1]:
organic_traffic = ['organic', 'referral', '(none)']

In [ ]:
def organic_1_and_pay_0_traffic(df: pd.DataFrame) -> pd.DataFrame:
    df['organic_traff'] = np.where(df['utm_medium'].isin(organic_traffic), 1, 0)
    return df

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from sqlalchemy import create_engine
import pandas as pd
import os
from datetime import datetime

# Функция для загрузки DataFrame из PKL в PostgreSQL
def load_pkl_to_postgres(file_path, table_name, db_config):
    # Загружаем DataFrame из файла PKL
    df = pd.read_pickle(file_path)

    # Устанавливаем соединение с PostgreSQL
    connection_string = f"postgresql://{db_config['username']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
    engine = create_engine(connection_string)

    # Загружаем DataFrame в таблицу PostgreSQL
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Данные успешно загружены в таблицу {table_name}.")

# Параметры подключения к базе данных
db_config = {
    'username': 'your_username',
    'password': 'your_password',
    'host': 'localhost',  # или IP-адрес вашего сервера
    'port': '5432',       # стандартный порт PostgreSQL
    'database': 'your_database'
}

# Путь к файлу PKL и имя таблицы
file_path = 'path/to/your/dataframe.pkl'  # Укажите путь к вашему файлу PKL
table_name = 'your_table_name'  # Укажите имя таблицы, в которую хотите загрузить данные

# Определяем DAG
with DAG('load_pkl_to_postgres_dag', start_date=datetime(2023, 1, 1), schedule_interval='@once', catchup=False) as dag:
    load_task = PythonOperator(
        task_id='load_pkl_task',
        python_callable=load_pkl_to_postgres,
        op_kwargs={
            'file_path': file_path,
            'table_name': table_name,
            'db_config': db_config
        }
    )

# Задача будет выполнена при запуске DAG